## Parameter Sweep Notebook ##
Sweeps two parameters (paramA, paramB) within the gait analysis algorithm over defined ranges, computes the mean and standard deviation of a chosen gait parameeter, and generates/saves heatmap visualizations. Details of the algorithm itself is  omitted due to sensitive information.

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# 1) Configuration
paramA_values = np.arange(-1.0, 8.0, 2.0)
paramB_values = np.arange(1.0, 8.0, 2.0)
target_metric  = 'stride_width'
unit           = 'cm'
config         = {'threshold': 34, 'calibration_enabled': True}

fig_dir = './figures'
os.makedirs(fig_dir, exist_ok=True)

runs = ['normal2', 'fast1', 'slow1']  # Run IDs

data_store = {
    'run1': None
}

# Step 2: Define processing function
def process_params(a, b, data):
    """
    Run a single sweep for parameters a, b on one dataset.
    Returns: a, b, mean_val, std_val
    """
    # 2.1 Actual Gait Analysis Algorithm (omitted)

    mean_val = None
    std_val  = None
    
    return a, b, mean_val, std_val


# 4) Execute sweep in parallel
all_stats = {}

for run_id, data in data_store.items():
    print(f"Sweeping parameters for {run_id}")

    # prepare result tables
    df_mean = pd.DataFrame(index=paramA_values, columns=paramB_values, dtype=float)
    df_std  = pd.DataFrame(index=paramA_values, columns=paramB_values, dtype=float)

    combos = list(product(paramA_values, paramB_values))
    results = Parallel(n_jobs=-1, verbose=5)(
        delayed(process_params)(a, b, data, config) for a, b in combos
    )

    for a, b, m, s in results:
        df_mean.loc[a, b] = m
        df_std.loc[a, b]  = s

    all_stats[run_id] = {'mean': df_mean, 'std': df_std}
    print(f"Finished {run_id}\n")

# 5) Plot heatmaps
for run_id, stats in all_stats.items():
    mat = stats['mean'].astype(float)
    plt.figure(figsize=(8,6))
    sns.heatmap(
        mat,
        annot=True, fmt='.2f',
        cmap='viridis',
        cbar_kws={'label': f"{target_metric} ({unit})"}
    )
    plt.gca().invert_yaxis()
    plt.title(f"{target_metric} Heatmap ({run_id})")
    plt.xlabel('paramA')
    plt.ylabel('paramB')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, f"{run_id}_heatmap.png"))
    plt.close()

print("All parameter sweeps complete.")